### SVM 
- Support Vector Machine
- 데이터프레임의 각 열은 열 벡터 형태로 구현 
- 열 벡터들이 각각 고유의 축을 갖는 벡터 공간을 만든다.
- 분석 대상이 되는 개별 관측값은 열 벡터에 관한 값을 해당 축의 좌표로 표시하여 벡터 공간에서의 위치를 나타냄
- 열 벡터가 2개 존재하는 데이터셋은 2차원 평면 공간에 좌표로 표시
- 열 벅테가 3개면 3차원 공간에 표시 
- SVM은 벡터 공간에 위치한 훈련 데이터의 좌표와 각 데이터가 어떤 분류 값을 가져야 하는지 정답을 입력 받아 학습

In [3]:
# Step-1 : 데이터 준비 / 기본 설정 

import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')

pd.set_option('display.max_columns', 15)

# Step-2 : 데이터 탐색 / 전처리
rdf = df.drop(['deck', 'embark_town'], axis = 1)
rdf = rdf.dropna(subset=['age'], how = 'any', axis = 0)

most_freq = rdf['embarked'].value_counts(dropna = True).idxmax()
rdf['embarked'].fillna(most_freq, inplace = True)

# Step-3 : 분석에 사용할 속성 선택
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis = 1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = 'town')

ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf.drop(['sex', 'embarked'], axis = 1, inplace = True)

# Step-4 : 데이터셋 구분 - 훈련용 / 검증용 

X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C', 'town_Q', 'town_S']]
y = ndf['survived']

from sklearn import preprocessing 
X = preprocessing.StandardScaler().fit(X).transform(X) 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

print('train 개수 : ', X_train.shape)
print('test 개수 : ', X_test.shape)

train 개수 :  (499, 9)
test 개수 :  (215, 9)


### 모형 학습 및 검증 
- sklearn에서 가져온 svm 모듈의 SVC() 함수를 사용하여 객체 생성 
- 데이터를 벡터 공간으로 매핑하는 함수를 kernel이라 함 
- kernel = 'rbf' 옵션으로 RBF(Radial Basis Function) 함수를 적용 
- train data를 fit() 메소드에 전달하여 모형 학습
- test data를 predict() 메소드에 입력하여 예측한 결과를 저장하고 실제 데이터(y_test)와 비교 

In [4]:
# Step-5 : SVM 분류 모형(sklearn) 사용

from sklearn import svm 

svm_model = svm.SVC(kernel='rbf')

svm_model.fit(X_train, y_train)

y_hat = svm_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 0 1 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


### 모형을 예측 능력을 평가하는 지표 계산
- confusion_matrix() 함수로 Confusion Matrix 계산
- 215명의 승객 중
    1. 미생존자를 정확히 예측한 TP : 120명
    2. 미생존자를 생존자로 잘못 분류한 FP : 5명
    3. 생존자를 미생존자로 잘못 분류한 FN : 35명
    4. 생존자를 정확하게 예측한 TN : 55명 
- classification_report() 함수로 precision, recall, f1 - score 지표를 출력

In [5]:
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(svm_matrix)
print('\n')

svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

[[120   5]
 [ 35  55]]


              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215

